In [ ]:
from __future__ import print_function
%matplotlib notebook
import pymp

import numpy as np
import matplotlib.pyplot as plt
import copy
import math
import cubic_spline_planner
import warnings
from functools import reduce

warnings.filterwarnings('ignore')
import time
from tqdm import tqdm_notebook as tqdm

SIM_LOOP = 130

# Parameter
MAX_SPEED = 3.5  # maximum speed [m/s]
MAX_LAT_ACCEL = 0.5  # maximum acceleration [m/ss]
MAX_LONG_ACCEL = 1.6  # maximum acceleration [m/ss]
MAX_CURVATURE = 1/5.0  # smaximum curvature [1/m]
MAX_ROAD_WIDTH = 2  # maximum road width [m]
D_ROAD_W = 1  # road width sampling length [m]
DT = 1  # time tick [s]
DT2 = 0.5
MAXT = 8  # max prediction time [m]
MINT = 2  # min prediction time [m]
TARGET_SPEED = 3.5  # target speed [m/s]
D_T_S = 0.75  # target speed sampling length [m/s]
N_S_SAMPLE = 3  # sampling number of target speed
ROBOT_RADIUS = 2.0  # robot radius [m]

# cost weights
KJ = 0.1
KT = 1
KD = 100
KLAT = 10.0
KLON = 1.0

show_animation = True

class quintic_polynomial:

    def __init__(self, xs, vxs, axs, xe, vxe, axe, T):
        # calc coefficient of quintic polynomial
        self.a0 = xs
        self.a1 = vxs
        self.a2 = axs / 2.0

        A = np.array([[T ** 3, T ** 4, T ** 5],
                      [3 * T ** 2, 4 * T ** 3, 5 * T ** 4],
                      [6 * T, 12 * T ** 2, 20 * T ** 3]])
        b = np.array([xe - self.a0 - self.a1 * T - self.a2 * T ** 2,
                      vxe - self.a1 - 2 * self.a2 * T,
                      axe - 2 * self.a2])
        x = np.linalg.solve(A, b)

        self.a3 = x[0]
        self.a4 = x[1]
        self.a5 = x[2]

    def calc_point(self, t):
        xt = self.a0 + self.a1 * t + self.a2 * t ** 2 + \
             self.a3 * t ** 3 + self.a4 * t ** 4 + self.a5 * t ** 5

        return xt

    def calc_first_derivative(self, t):
        xt = self.a1 + 2 * self.a2 * t + \
             3 * self.a3 * t ** 2 + 4 * self.a4 * t ** 3 + 5 * self.a5 * t ** 4

        return xt

    def calc_second_derivative(self, t):
        xt = 2 * self.a2 + 6 * self.a3 * t + 12 * self.a4 * t ** 2 + 20 * self.a5 * t ** 3

        return xt

    def calc_third_derivative(self, t):
        xt = 6 * self.a3 + 24 * self.a4 * t + 60 * self.a5 * t ** 2

        return xt


class quartic_polynomial:

    def __init__(self, xs, vxs, axs, vxe, axe, T):
        # calc coefficient of quartic polynomial

        self.a0 = xs
        self.a1 = vxs
        self.a2 = axs / 2.0

        A = np.array([[3 * T ** 2, 4 * T ** 3],
                      [6 * T, 12 * T ** 2]])
        b = np.array([vxe - self.a1 - 2 * self.a2 * T,
                      axe - 2 * self.a2])
        x = np.linalg.solve(A, b)

        self.a3 = x[0]
        self.a4 = x[1]

    def calc_point(self, t):
        xt = self.a0 + self.a1 * t + self.a2 * t ** 2 + \
             self.a3 * t ** 3 + self.a4 * t ** 4

        return xt

    def calc_first_derivative(self, t):
        xt = self.a1 + 2 * self.a2 * t + \
             3 * self.a3 * t ** 2 + 4 * self.a4 * t ** 3

        return xt

    def calc_second_derivative(self, t):
        xt = 2 * self.a2 + 6 * self.a3 * t + 12 * self.a4 * t ** 2

        return xt

    def calc_third_derivative(self, t):
        xt = 6 * self.a3 + 24 * self.a4 * t

        return xt


class Frenet_path:
    def __init__(self, t=None, d=None, d_d=None, d_dd=None, d_ddd=None, s=None, s_d=None, s_dd=None, s_ddd=None, cd=0,
                 cv=0, cf=0, x=None, y=None, yaw=None, ds=None, c=None):
        self.t = t or []
        self.d = d or []
        self.d_d = d_d or []
        self.d_dd = d_dd or []
        self.d_ddd = d_ddd or []
        self.s = s or []
        self.s_d = s_d or []
        self.s_dd = s_dd or []
        self.s_ddd = s_dd or []
        self.cd = cd or 0
        self.cv = cv or 0
        self.cf = cf or 0

        self.x = x or []
        self.y = y or []
        self.yaw = yaw or []
        self.ds = ds or []
        self.c = c or []

    def __add__(self, o):
        return Frenet_path(
            self.t + o.t,
            self.d + o.d,
            self.d_d + o.d_d,
            self.d_dd + o.d_dd,
            self.d_ddd + o.d_ddd,
            self.s + o.s,
            self.s_d + o.s_d,
            self.s_dd + o.s_dd,
            self.s_ddd + o.s_ddd,
            self.cd + o.cd,
            self.cv + o.cv,
            self.cf + o.cf,

            self.x + o.x,
            self.y + o.y,
            self.yaw + o.yaw,
            self.ds + o.ds,
            self.c + o.c)


def calc_frenet_path(c_speed, c_acc, c_d, c_d_d, c_d_dd, s0, Ti, di, tv):
    fp = Frenet_path()
    lat_qp = quintic_polynomial(c_d, c_d_d, c_d_dd, di, 0.0, 0.0, Ti)

    fp.t = [t for t in np.linspace(0.0, Ti, round(Ti / DT2) + 1)]
    fp.d = [lat_qp.calc_point(t) for t in fp.t]
    fp.d_d = [lat_qp.calc_first_derivative(t) for t in fp.t]
    fp.d_dd = [lat_qp.calc_second_derivative(t) for t in fp.t]
    fp.d_ddd = [lat_qp.calc_third_derivative(t) for t in fp.t]

    lon_qp = quartic_polynomial(s0, c_speed, c_acc, tv, 0.0, Ti)

    fp.s = [lon_qp.calc_point(t) for t in fp.t]
    fp.s_d = [lon_qp.calc_first_derivative(t) for t in fp.t]
    fp.s_dd = [lon_qp.calc_second_derivative(t) for t in fp.t]
    fp.s_ddd = [lon_qp.calc_third_derivative(t) for t in fp.t]

    Jp = np.mean(np.power(fp.d_ddd, 2))  # square of jerk
    Js = np.mean(np.power(fp.s_ddd, 2))  # square of jerk

    # square of diff from target speed
    ds = (TARGET_SPEED - fp.s_d[-1]) ** 2

    fp.cd = KJ * Jp + KT * Ti + KD * fp.d[-1] ** 2
    fp.cv = KJ * Js + KT * Ti + KD * ds
    fp.cf = KLAT * fp.cd + KLON * fp.cv

    return fp


def calc_path_segment(t, d, v, c_speed, c_acc, c_d, c_d_d, c_d_dd, s0, ob, csp):
    fp = calc_frenet_path(c_speed, c_acc, c_d, c_d_d, c_d_dd, s0, t, d, v)
    calc_global_path(fp, csp)
    if not check_path(fp, ob):
        fp.cf = np.inf
    return fp


def calc_optimal_path(c_speed, c_acc, c_d, c_d_d, c_d_dd, s0, ob, csp):
    d_vec = np.linspace(-MAX_ROAD_WIDTH, MAX_ROAD_WIDTH, round(2 * MAX_ROAD_WIDTH / D_ROAD_W + 1))
    #d_vec = [0]
    v_vec = np.linspace(TARGET_SPEED - D_T_S * N_S_SAMPLE, TARGET_SPEED + D_T_S * N_S_SAMPLE, 2 * N_S_SAMPLE + 1)

    dyn_prog = [[[{'cost': np.inf, 'path': [], 's': s0} for e1 in range(len(v_vec))] for e2 in range(len(d_vec))] for e3 in range(round(MAXT / DT + 1))]
    for d_ind, d in enumerate(d_vec):
        for v_ind, v in enumerate(v_vec):
            dyn_prog[0][d_ind][v_ind]['cost'] = 0


    
    for dyn_ind in range(MINT, round(MAXT / DT + 1)):
        ex_array = pymp.shared.list(range(len(d_vec) * len(v_vec)))
#         ex_array = list(range(len(d_vec) * len(v_vec)))
        with pymp.Parallel(6) as p:
#         for _ in range(1):
            for d2_ind in p.range(0, len(d_vec)):
#             for d2_ind in range(0, len(d_vec)):
                d2 = d_vec[d2_ind]
                for v2_ind, v2 in enumerate(v_vec):
                    min_cost = np.inf
                    optim_path = []
                    optim_s = 0
                    for t_ind in range(0, dyn_ind - MINT + 1):
                        for d1_ind, d1 in enumerate(d_vec):
                            for v1_ind, v1 in enumerate(v_vec):
                                dyn_s0 = dyn_prog[t_ind][d1_ind][v1_ind]['s']
                                if t_ind == 0:
                                    fp = calc_path_segment((dyn_ind - t_ind) * DT, d2, v2, c_speed, c_acc, c_d, c_d_d,
                                                           c_d_dd, dyn_s0, ob, csp)
                                else:
                                    fp = calc_path_segment((dyn_ind - t_ind) * DT, d2, v2, v1, 0, d1, 0, 0, dyn_s0, ob, csp)

                                new_cost = dyn_prog[t_ind][d1_ind][v1_ind]['cost'] + fp.cf
                                if new_cost < min_cost:
                                    min_cost = new_cost
                                    optim_path = dyn_prog[t_ind][d1_ind][v1_ind]['path'] + [fp]
                                    optim_s = fp.s[-1]

                    ex_array[d2_ind + v2_ind * len(d_vec)] = (min_cost, optim_path, optim_s)
                                
        for d2_ind in range(len(d_vec)):
            for v2_ind in range(len(v_vec)):
                dyn_prog[dyn_ind][d2_ind][v2_ind]['cost'] = ex_array[d2_ind + v2_ind * len(d_vec)][0]
                dyn_prog[dyn_ind][d2_ind][v2_ind]['path'] = ex_array[d2_ind + v2_ind * len(d_vec)][1]
                dyn_prog[dyn_ind][d2_ind][v2_ind]['s'] = ex_array[d2_ind + v2_ind * len(d_vec)][2]
        
    final_min_cost = np.inf
    best_path = None
    for d_ind, d in enumerate(d_vec):
        for v_ind, v in enumerate(v_vec):
            if dyn_prog[-1][d_ind][v_ind]['cost'] < final_min_cost:
                final_min_cost = dyn_prog[-1][d_ind][v_ind]['cost']
                best_path = dyn_prog[-1][d_ind][v_ind]['path']

    return reduce((lambda x,y:x+y), best_path)


def unwrap_angle(x):
    x = np.fmod(x + np.pi, 2 * np.pi)
    if x < 0:
        x += 2 * np.pi
    return x - np.pi


def calc_global_path(fp, csp):
    for i in range(len(fp.s)):
        ix, iy = csp.calc_position(fp.s[i])
        if ix is None:
            break
        iyaw = csp.calc_yaw(fp.s[i])
        di = fp.d[i]
        fx = ix + di * math.cos(iyaw + math.pi / 2.0)
        fy = iy + di * math.sin(iyaw + math.pi / 2.0)
        fp.x.append(fx)
        fp.y.append(fy)

    # calc yaw and ds
    for i in range(len(fp.x) - 1):
        dx = fp.x[i + 1] - fp.x[i]
        dy = fp.y[i + 1] - fp.y[i]
        fp.yaw.append(math.atan2(dy, dx))
        fp.ds.append(math.hypot(dx, dy))

    fp.yaw.append(fp.yaw[-1])
    fp.ds.append(fp.ds[-1])

    # calc curvature
    for i in range(len(fp.yaw) - 1):
        fp.c.append(csp.calc_curvature(fp.s[i]))


def check_collision(fp, ob):
    for i in range(len(ob[:, 0])):
        d = [((ix - ob[i, 0]) ** 2 + (iy - ob[i, 1]) ** 2)
             for (ix, iy) in zip(fp.x, fp.y)]

        collision = any([di <= ROBOT_RADIUS ** 2 for di in d])

        if collision:
            return False

    return True


def check_path(fp, ob):
    if any([v > MAX_SPEED for v in fp.s_d]):  # Max speed check
        return False
    elif any([abs(a) > MAX_LONG_ACCEL for a in fp.s_dd]):  # Max accel check
        return False
    elif any([abs(v * v * c) > MAX_LAT_ACCEL for (v, c) in zip(fp.s_d, fp.c)]):  # Max accel check
        return False
    elif any([abs(c) > MAX_CURVATURE for c in fp.c]):  # Max curvature check
        return False
    elif not check_collision(fp, ob):
        return False

    return True


def generate_target_course(x, y):
    csp = cubic_spline_planner.Spline2D(x, y)
    s = np.arange(0, csp.s[-1], 0.1)

    rx, ry, ryaw, rk = [], [], [], []
    for i_s in s:
        ix, iy = csp.calc_position(i_s)
        rx.append(ix)
        ry.append(iy)
        ryaw.append(csp.calc_yaw(i_s))
        rk.append(csp.calc_curvature(i_s))

    return rx, ry, ryaw, rk, csp


# way points
wx = [14.5516988114666, 14.5161763774167, 14.4971884213034, 14.4932254727246, 14.5028761596214, 14.5248222964927, 14.5578341724164, 14.6007660322393, 14.6525517444671, 14.712200649561, 14.7787935825122, 14.8514790637307, 14.9294696524456, 15.0120384569722, 15.0985157963547, 15.1882860080464, 15.2807843964368, 15.3754943171804, 15.4719443924244, 15.5697058521728, 15.6683899971603, 15.7676457787426, 15.8671574914419, 15.966642573913, 16.0658495142225, 16.1645558554536, 16.2625662977721, 16.3597108932033, 16.4558433294868, 16.5508392994855, 16.6445949527386, 16.7370254258503, 16.8280634485153, 16.9176580220803, 17.005773167645, 17.0923867407975, 17.1774893101795, 17.2610830971676, 17.3431809740453, 17.4238055181325, 17.5029881194221, 17.5807681393594, 17.657192118481, 17.7323130307094, 17.8061895821777, 17.8788855525348, 17.9504691767545, 18.0210125655442, 18.0905911625185, 18.1592832363719, 18.227169406349, 18.2943321993779, 18.3608556372912, 18.4268248526246, 18.4923257315377, 18.5574445824616, 18.6222678291335, 18.6868817267313, 18.7513720998757, 18.8158241013162, 18.8803219901686, 18.9449489286192, 19.0097867960555, 19.0749160196321, 19.1404154203195, 19.2063620735301, 19.2728311834532, 19.3398959702717, 19.4076275694732, 19.4760949425008, 19.5453647980297, 19.6155015231863, 19.686567124062, 19.7586211749051, 19.8317207754056, 19.9059205155182, 19.9812724472952, 20.0578260632326, 20.1356282806558, 20.2147234316991, 20.2951532584577, 20.3769569129135, 20.4601709612599, 20.5448293922733, 20.6309636293978, 20.7186025462315, 20.8077724851227, 20.8984972785999, 20.9907982733806, 21.0846943567183, 21.1802019848646, 21.2773352134375, 21.3761057295029, 21.4765228851905, 21.5785937326755, 21.6823230603752, 21.7877134302172, 21.8947652158488, 22.0034766416701, 22.1138438225787, 22.2258608043288, 22.3395196044136, 22.4548102533874, 22.571720836557, 22.6902375359723, 22.8103446726602, 22.9320247490472, 23.0552584915252, 23.1800248931203, 23.3063012562288, 23.4340632353916, 23.5632848800792, 23.6939386774687, 23.8259955951934, 23.9594251240527, 24.0941953206704, 24.2302728500952, 24.3676230283368, 24.5062098648374, 24.6459961048755, 24.7869432719044, 24.92901170983, 25.0721606252293, 25.2163481295168, 25.3615312810665, 25.5076661272922, 25.6547077467005, 25.8026102909178, 25.9513270267056, 26.1008103779674, 26.2510119677601, 26.4018826603148, 26.5533726030767, 26.7054312687707, 26.8580074974993, 27.0110495388802, 27.164505094228, 27.3183213587858, 27.472445064006, 27.6268225198917, 27.781399657392, 27.9361220708549, 28.0909350605397, 28.2457836751835, 28.4006127546201, 28.5553669724476, 28.7099908787421, 28.8644289428022, 29.0186255959283, 29.1725252742161, 29.3260724613627, 29.4792117314687, 29.6318877918204, 29.7840455256491, 29.9356300348367, 30.0865866825646, 30.2368611358748, 30.3863994081378, 30.5351479013947, 30.6830534485612, 30.8300633554594, 30.9761254426724, 31.1211880871764, 31.2652002637361, 31.4081115860395, 31.5498723475349, 31.6904335619499, 31.8297470034607, 31.9677652464789, 32.1044417050384, 32.2397306717293, 32.3735873561672, 32.5059679229636, 32.6368295291484, 32.7661303610397, 32.8938296704956, 33.0198878105454, 33.1442662703519, 33.2669277094658, 33.3878359913469, 33.5069562161316, 33.624254752579, 33.7396992692032, 33.853258764522, 33.9649035964181, 34.0746055105677, 34.1823376678973, 34.2880746710532, 34.391792589849, 34.493468985644, 34.5930829346497, 34.6906150501123, 34.7860475033565, 34.8793640436579, 34.970550016916, 35.0595923831076, 35.1464797324931, 35.2312023005441, 35.3137519815793, 35.3941223410935, 35.4723086267333, 35.5483077779319, 35.622118434155, 35.693740941768, 35.7631773594796, 35.8304314623844, 35.8955087445489, 35.9584164201684, 36.0191634232492, 36.0777604058327, 36.1342197347588, 36.188555486914, 36.2407834430317, 36.2909210799828, 36.3389875615852, 36.3850037279242, 36.4289920831603, 36.4709767818954, 36.5109836140255, 36.5490399881269, 36.585174913367, 36.6194189799943, 36.651804338305, 36.6823646762647, 36.7111351956349, 36.7381525867507, 36.763455001845, 36.7870820270819, 36.809074653165, 36.8294752446463, 36.8483275079621, 36.8656764581041, 36.8815683841281, 36.8960508133495, 36.9091724744127, 36.9209832591167, 36.9315341831757, 36.9408773457602, 36.9490658881568, 36.9561539511407, 36.9621966315529, 36.9672499378112, 36.9713707444998, 36.9746167461234, 36.9770464099348, 36.9787189280348, 36.9796941686766, 36.9800326268158, 36.9797953740508, 36.9790440078873, 36.9778406004595, 36.9762476466901, 36.9743280118849, 36.9721448790861, 36.9697616958269, 36.9672421206315, 36.9646499692375, 36.9620491605552, 36.9595036624179, 36.9570774372038, 36.9548343873763, 36.9528383009619, 36.9511527970376, 36.9498412713743, 36.9489668420201, 36.9485922951754, 36.9487800313142, 36.9495920114675, 36.9510897038508, 36.9533340310185, 36.9563853171149, 36.9603032360235, 36.9651467596665, 36.9709741070362, 36.977842693964, 36.9858090833104, 36.9949289359958, 37.0052569629695, 37.0168468776093, 37.0297513494174, 37.0440219581956, 37.0597091496471, 37.076862191349, 37.0955291304544, 37.1157567520315, 37.1375905384708, 37.1610746304324, 37.1862517886933, 37.2131633572284, 37.2418492276694, 37.2723478051087, 37.3046959750961, 37.3389290720438, 37.3750808491803, 37.4131834497472, 37.4532673797538, 37.4953614821054, 37.5394929126148, 37.5856871169853, 37.6339678096553, 37.6843569544238, 37.7368747462397, 37.7915395950032, 37.8483681106698, 37.9073750902513, 37.9685735062618, 38.0319744970743, 38.0975873588136, 38.1654195390277, 38.2354766317058, 38.3077623745575, 38.3822786475871, 38.4590254735831, 38.5380010199408, 38.619201602898, 38.7026216926984, 38.7882539207108, 38.8760890887729, 38.9661161789699, 39.0583223665888, 39.1526930334915, 39.2492117835446, 39.34786046024, 39.4486191645937, 39.5514662761405, 39.6563784744488, 39.7633307623163, 39.872296490884, 39.9832473862204, 40.096153576401, 40.2109836218421, 40.3277045449321, 40.4462818625597, 40.5666796195866, 40.6888604232904, 40.8127854792316, 40.9384146286864, 41.0657063869505, 41.1946179826523, 41.325105398248, 41.4571234127255, 41.5906256427, 41.7255645875646, 41.8618916733828, 41.9995572985131, 42.1385108800474, 42.2787009016733, 42.4200749596995, 42.5625798142026, 42.7061614362006, 42.8507650588001, 42.9963352271051, 43.1428158493952, 43.2901502480324, 43.4382812116282, 43.5871510473227, 43.7367016325745, 43.886874468453, 44.0376107313838, 44.1888513277185, 44.3405369450871, 44.4926081060241, 44.6450052217244, 44.7976686432167, 44.9505387153078, 45.1035558290524, 45.2566604734654, 45.4097932879442, 45.5628951130413, 45.7159070428275, 45.8687704749312, 46.0214271601989, 46.1738192531636, 46.3258893606354, 46.4775805895218, 46.628836595645, 46.7796016288894, 46.9298205810314, 47.0794390290084, 47.2284032799516, 47.3766604152508, 47.5241583305506, 47.6708457796595, 47.8166724126656, 47.9615888156234, 48.105546548448, 48.2484981808995, 48.3903973291966, 48.531198688533, 48.6708580667242, 48.8093324156315, 48.9465798612171, 49.0825597319555, 49.2172325859406, 49.3505602376418, 49.4825057801761, 49.6130336098784, 49.7421094468534, 49.8697003549984, 49.9957747600075, 50.1203024645246, 50.2432546660761, 50.3646039670185, 50.4843243885694, 50.6023913799668, 50.7187818270189, 50.8334740593848, 50.9464478548477, 51.0576844432323, 51.1671665078139, 51.2748781860245, 51.3808050681054, 51.4849341943499, 51.5872540478022, 51.6877545513351, 51.7864270578592, 51.8832643398416, 51.978260580966, 52.0714113606854, 52.1627136408291, 52.2521657500618, 52.3397673660824, 52.4255194964579, 52.5094244601686, 52.5914858635462, 52.6717085784493, 52.7500987169114, 52.8266636049601, 52.9014117589168, 52.9743528511905, 53.0454976858474, 53.1148581625183, 53.1824472497833, 53.2482789488619, 53.3123682569028, 53.3747311389373, 53.4353844838287, 53.4943460717003, 53.5516345326876, 53.607269310029, 53.6612706218449, 53.7136594147674, 53.764457326289, 53.8136866453637, 53.8613702653303, 53.9075316417131, 53.9521947480311, 53.9953840379224, 54.0371243901306, 54.077441070746, 54.1163596880166, 54.1539061464535, 54.1901065936797, 54.2249873901174, 54.2585750498351, 54.2908962002038, 54.3219775397929, 54.3518457814448, 54.380527624486, 54.4080496920685, 54.4344384970945, 54.4597203950165, 54.4839215437741, 54.5070678476331, 54.529184926653, 54.5502980770261, 54.5704322152573, 54.5896118452854, 54.6078610208554, 54.6252032989487, 54.6416617077053, 54.6572587032628, 54.6720161362949, 54.6859552152058, 54.6990964655397, 54.7114597085073, 54.7230640193516, 54.7339276937595, 54.7440682274189, 54.7535022717684, 54.7622456288595, 54.770313196932, 54.777718971097, 54.7844759969706, 54.7905963731647, 54.7960912059377, 54.8009706029945, 54.8052436586757, 54.8089184287276, 54.8120019134314, 54.8145000599635, 54.8164177381319, 54.8177587312316, 54.818525730457, 54.8187203314835, 54.8183430187771, 54.8173931759761, 54.8158690718575, 54.8137678743687, 54.8110856357941, 54.8078173059988, 54.803956738217, 54.7994966974725, 54.7944288571623, 54.7887438234277, 54.7824311365504, 54.7754792933086, 54.7678757505021, 54.7596069560279, 54.7506583579233, 54.7410144226431, 54.7306586627867, 54.7195736580715, 54.7077410735175, 54.6951417002777, 54.6817554619656, 54.6675614653928, 54.6525380153916, 54.6366626484009, 54.6199121718952, 54.6022626932117, 54.5836896619342, 54.5641678862135, 54.5436715990476, 54.5221744762697, 54.499649680218, 54.4760699003682, 54.4514074093914, 54.4256340776827, 54.3987214367479, 54.3706407256617, 54.3413629167459, 54.3108587814881, 54.2790989089277, 54.2460538005868, 54.2116938566883, 54.1759894729535, 54.1389110564591, 54.1004290959632, 54.0605141857335, 54.0191370961409, 53.9762688090369, 53.9318805746697, 53.8859439506785, 53.8384308476074, 53.7893135900182, 53.7385649545259, 53.6861582202422, 53.6320672118968, 53.5762663433871, 53.5187306753012, 53.4594359564158, 53.398358650369, 53.335476000899, 53.2707660576554, 53.2042077445427, 53.1357808627196, 53.0654661643237, 52.9932453828468, 52.9191012436352, 52.843017545931, 52.7649791510119, 52.684972070515, 52.6029834443266, 52.5190015980389, 52.4330160810399, 52.3450176842964, 52.2549984529532, 52.162951746824, 52.0688722291278, 51.9727558991621, 51.8746001336134, 51.774403665082, 51.6721666460561, 51.5678906237562, 51.4615785579924, 51.3532348538803, 51.2428653743381, 51.1304773856785, 51.0160796716603, 50.8996824276216, 50.7812973350771, 50.6609375127409, 50.5386175581763, 50.4143534939293, 50.2881628250553, 50.1600644591231, 50.030078762747, 49.8982274845689, 49.7645337747363, 49.6290221961649, 49.4917186501379, 49.352650373196, 49.2118459722753, 49.0693352749491, 48.9251494437833, 48.7793208486469, 48.6318830970877, 48.4828709330905, 48.3323202995249, 48.1802682233366, 48.026752800788, 47.8718131896101, 47.7154895168737, 47.5578228625086, 47.398855260227, 47.2386295589476, 47.0771894641169, 46.9145794392067, 46.7508446648804, 46.5860310310284, 46.4201849949656, 46.2533536570904, 46.085584607361, 45.9169258749215, 45.7474259610872, 45.5771336668169, 45.4060981034749, 45.2343687043135, 45.0619949675105, 44.8890266423061, 44.7155134458764, 44.5415052106609, 44.3670516444914, 44.1922023917549, 44.0170069371951, 43.8415145657147, 43.6657742573856, 43.4898346797727, 43.3137440866647, 43.1375502947831, 42.9613006649031, 42.7850419145726, 42.6088202674116, 42.4326811098109, 42.2566693042292, 42.0808287507117, 41.9052026493876, 41.7298333382701, 41.5547621251686, 41.3800294308, 41.2056746365998, 41.0317360758738, 40.8582509197584, 40.6852552710219, 40.5127840039972, 40.3408707674441, 40.1695479325308, 39.9988465954621, 39.8287965298071, 39.6594261303808, 39.4907624327946, 39.322830990908, 39.1556560065181, 38.9892601250774, 38.8236646006647, 38.6588891273918, 38.4949518828445, 38.3318695271774, 38.1696571813961, 38.0083284616829, 37.8478953008028, 37.6883682114461, 37.5297560748877, 37.3720662489065, 37.2153044983409, 37.0594750233375, 36.9045805666055, 36.7506223005151, 36.5975997899357, 36.44551124304, 36.2943533120993, 36.1441211499726, 35.9948086233658, 35.8464079638125, 35.6989101959106, 35.5523048781623, 35.4065803657103, 35.2617235831977, 35.1177202931434, 34.9745550171594, 34.8322110516417, 34.6906706063754, 34.5499148804725, 34.4099238528163, 34.2706766785894, 34.1321513729954, 33.9943252285768, 33.8571746038946, 33.7206750911187, 33.5848014975187, 33.4495279943863, 33.3148281055194, 33.1806748021934, 33.0470404153068, 32.9138970553498, 32.7812162908341, 32.648969229543, 32.517127003613, 32.3856602403833, 32.2545396189804, 32.1237355596817, 31.9932186332657, 31.8629592143909, 31.7329279852114, 31.6030955313208, 31.4734328606873, 31.3439111472125, 31.2145018489454, 31.0851768800344, 30.9559085243397, 30.8266696142648, 30.6974333367416, 30.5681738245333, 30.4388654123389, 30.3094834950536, 30.1800038531458, 30.0504033858988, 29.9206596461856, 29.7907510144884, 29.6606568283774, 29.5303573810101, 29.3998338881487, 29.2690686601532, 29.138044832984, 29.0067468916329, 28.8751601888016, 28.7432711784376, 28.6110676768284, 28.4785383155141, 28.3456732145258, 28.2124632780071, 28.0789009663595, 27.94497968674, 27.8106941595365, 27.6760401567356, 27.5410146663181, 27.4056159740737, 27.2698434109629, 27.133697346015, 26.9971795406002, 26.860292750676, 26.7230406529283, 26.5854283616574, 26.4474618361232, 26.3091479982796, 26.1704947666094, 26.031511190687, 25.8922069119862, 25.7525929078482, 25.612680561508, 25.4724821772628, 25.3320109499914, 25.191280546177, 25.05030577016, 24.9091010847834, 24.7676825423988, 24.6260659806638, 24.4842680922867, 24.3423056839463, 24.2001960828779, 24.0579566625447, 23.915605248484, 23.7731597433551, 23.630638148679, 23.4880581433377, 23.3454379190004, 23.2027954047949, 23.0601483403403, 22.9175143448693, 22.7749103779035, 22.6323536354689, 22.4898607627572, 22.3474478197009, 22.2051306514171, 22.0629242360593, 21.9208433764794, 21.7789018644827, 21.6371132149667, 21.4954897472336, 21.3540435628877, 21.2127858995779, 21.0717264595137, 20.9308752276865, 20.7902404546196, 20.6498297811503, 20.5096501176957, 20.3697071981789, 20.230005740819, 20.0905498608691, 19.9513424153891, 19.8123853907078, 19.6736797938969, 19.5352258878067, 19.3970226719433, 19.2590684269273, 19.121360563507, 18.9838955782359, 18.8466690570952, 18.7096759446065, 18.5729102392681, 18.4363654037786, 18.3000340181179, 18.1639082029445, 18.0279795633446, 17.8922392060829, 17.7566775402733, 17.6212848242539, 17.4860511052419, 17.3509661956969, 17.216019360574, 17.081200214769, 16.9464983557095, 16.8119035163467, 16.6774056646831, 16.5429948066001, 16.4086618648748, 16.2743975833747, 16.1401941740775, 16.006043507653, 15.8719392883177, 15.7378754839802, 15.6038468744974, 15.469850231701, 15.335883001472, 15.2019435584558, 15.0680326499137, 14.9341518440268, 14.800304137643, 14.666494767469, 14.5327310787575, 14.3990208568269, 14.265375428718, 14.1318075368874, 13.9983317211135, 13.864965436749, 13.7317276447212, 13.5986402611772, 13.4657268051709, 13.3330144342872, 13.2005317729975, 13.0683094513826, 12.9363823402884, 12.804786344963, 12.6735606757665, 12.5427464733082, 12.4123883924665, 12.2825322925501, 12.153227323708, 12.0245254385255, 11.8964800854188, 11.7691474453826, 11.6425858807902, 11.5168565919754, 11.3920221118178, 11.2681473285443, 11.1452984039885, 11.0235441567255, 10.90295517315, 10.7836023188936, 10.6655595029201, 10.548900488142, 10.4336998049408, 10.3200351061535, 10.2079817764288, 10.0976178064985, 9.98902026716103, 9.88226776035112, 9.77743680386638, 9.67460498686274, 9.57384920748397, 9.47524353709487, 9.37886459517987, 9.28478538351979, 9.19307656895585, 9.10380917698459, 9.01705067510992, 8.9328672347809, 8.85132165176315, 8.77247478891381, 8.69638283325899, 8.62310237110495, 8.55268237946975, 8.4851706073314, 8.42061038726369, 8.35903994370632, 8.30049569391428, 8.24500710107519, 8.1926008428984, 8.14329812986287, 8.09711409100276, 8.05406075168992, 8.01414338095503, 7.97736437783548, 7.94371861382246, 7.91319680404502, 7.88578393276686, 7.86145928265737, 7.84019823451863, 7.82196971420292, 7.80673906757025, 7.79446336923026, 7.78509863570818, 7.7785937724414, 7.77489410873659, 7.77394040220031, 7.77566844707867, 7.78001244399008, 7.78690147092243, 7.79626089397167, 7.80801388610204, 7.82208278006508, 7.8383864416836, 7.8568396391447, 7.87735884670704, 7.89986193548313, 7.92426124639489, 7.95047288058766, 7.9784138661805, 8.00799937316156, 8.03915025175283, 8.07178701848918, 8.1058328883212, 8.14121778886238, 8.17787168973656, 8.21572902814016, 8.25473272176365, 8.29482759210397, 8.33596779638613, 8.37810942571102, 8.42121585691689, 8.46526156912364, 8.5102258237871, 8.55609187438414, 8.60285795896245, 8.6505237846918, 8.69909883636637, 8.74860404064546, 8.79906117048372, 8.85050448119601, 8.9029771497874, 8.9565235478188, 9.01120109230565, 9.06706739446724, 9.12419017865954, 9.18263674625315, 9.24248386037451, 9.30380550593263, 9.36668115247305, 9.43119197706941, 9.49741283994198, 9.56541794864242, 9.63528055888318, 9.70707042959725, 9.78084088634361, 9.8566428583162, 9.93452012508115, 10.0144910519687, 10.0965770793125, 10.1807700402827, 10.2670477489241, 10.3553703278705, 10.4456693480233, 10.537861054037, 10.6318277027319, 10.7274388518126, 10.8245244768322, 10.922890608749, 11.0223139250999, 11.1225515512514, 11.2233220899296, 11.3243232434133, 11.4252343245721, 11.5257016817884, 11.6253652357218, 11.7238446599962, 11.820763321066, 11.9157305958729, 12.0083741315195, 12.0983455318238, 12.1853205188377, 12.2690232349232, 12.3492426247012, 12.4258418609669, 12.498798038434, 12.568209656419, 12.6343232243851, 12.6975833717409, 12.7586482274681, 12.8184349359915, 12.8781684730786, 12.9394212944227, 13.0041741491201, 13.0748678898761, 13.154476361417, 13.2465809765031, 13.3554413242736, 13.4860733353962, 13.6443782767712, 13.837211565531]
wy = [34.9987385766254, 34.959465423681, 34.9531481320604, 34.9762148562436, 35.0253661456865, 35.0975595930784, 35.1899951394889, 35.3001010148212, 35.4255202925221, 35.5640980380301, 35.713869030958, 35.8730460415159, 36.0400086421802, 36.2132925361029, 36.3915793842385, 36.5736871136357, 36.7585606898061, 36.9452633365359, 37.1329681869529, 37.3209503500967, 37.5085793776731, 37.6953121160884, 37.8806859292774, 38.0643122782389, 38.2458706435913, 38.4251027778476, 38.6018072744898, 38.7758344412967, 38.9470814657415, 39.115487860636, 39.2810311785453, 39.4437229838427, 39.6036050716064, 39.7607459228912, 39.9152373862268, 40.0671915755109, 40.2167379747704, 40.3640207405657, 40.5091961931078, 40.652430487443, 40.7938974563422, 40.9337766168069, 41.0722513323696, 41.2095071236303, 41.3457301197257, 41.4811056436773, 41.6158169248079, 41.7500439316555, 41.8839623190425, 42.0177424831883, 42.1515487189709, 42.2855384736607, 42.419861691657, 42.5546602449667, 42.6900674443572, 42.8262076263168, 42.9631958111396, 43.1011374276385, 43.2401281001689, 43.380253493818, 43.5215892137881, 43.664200755162, 43.8081434994038, 43.9534627541014, 44.1001938326082, 44.2483621703919, 44.3979834750347, 44.5490639069764, 44.7016002882178, 44.8555803363388, 45.0109829213081, 45.1677783426865, 45.3259286249433, 45.4853878287204, 45.6461023759914, 45.8080113871696, 45.9710470283235, 46.13513486676, 46.3001942333324, 46.4661385899241, 46.6328759006522, 46.8003090054198, 46.9683359945334, 47.1368505831821, 47.305742484655, 47.4748977812481, 47.6441992918855, 47.8135269355524, 47.9827580896992, 48.151767942848, 48.3204298406906, 48.4886156250278, 48.6561959649602, 48.8230406797898, 48.9890190531521, 49.154000137942, 49.3178530516488, 49.4804472617623, 49.6416528609521, 49.8013408317678, 49.9593833006489, 50.1156537810628, 50.2700274056397, 50.4223811471919, 50.5725940285503, 50.7205473211729, 50.8661247325131, 51.009212582168, 51.1496999668364, 51.2874789141635, 51.422444525552, 51.5544951080516, 51.6835322954593, 51.809461158771, 51.9321903061579, 52.0516319726453, 52.1677020996872, 52.2803204048567, 52.3894104418589, 52.4948996511172, 52.5967194011677, 52.6948050211128, 52.7890958244105, 52.8795351242463, 52.9660702407915, 53.0486525006007, 53.1272372284637, 53.2017837319718, 53.2722552791273, 53.33861906926, 53.4008461975651, 53.4589116135687, 53.5127940737992, 53.5624760889894, 53.6079438660837, 53.6491872453702, 53.6861996330176, 53.7189779293212, 53.747522452943, 53.7718368614354, 53.7919280683311, 53.8078061570882, 53.8194842921495, 53.8269786274057, 53.83030821232, 53.8294948959716, 53.8245632292752, 53.8155403656393, 53.8024559602817, 53.7853420684692, 53.7642330428768, 53.7391654303386, 53.7101778681349, 53.6773109801277, 53.6406072728396, 53.6001110317468, 53.5558682179531, 53.50792636542, 53.4563344789191, 53.4011429329111, 53.3424033714591, 53.2801686093785, 53.2144925347249, 53.1454300127772, 53.0730367916589, 52.9973694096837, 52.9184851045672, 52.8364417246167, 52.7512976419665, 52.6631116679856, 52.5719429709221, 52.4778509958702, 52.380895387148, 52.2811359131027, 52.1786323934688, 52.0734446292418, 51.9656323352446, 51.8552550752722, 51.7423721999499, 51.6270427873154, 51.5093255860834, 51.3892789617545, 51.2669608453455, 51.1424286849793, 51.0157394002188, 50.8869493390899, 50.7561142379213, 50.6232891838496, 50.4885285800572, 50.3518861136733, 50.2134147263172, 50.0731665872613, 49.9311930691703, 49.7875447263597, 49.6422712755566, 49.4954215791155, 49.3470436305518, 49.1971845425236, 49.045890536952, 48.8932069375259, 48.739178164157, 48.5838477297425, 48.4272582387837, 48.2694513880181, 48.1104679689337, 47.9503478722039, 47.7891300935481, 47.6268527416762, 47.4635530474947, 47.2992673749457, 47.1340312333475, 46.9678792909159, 46.8008453897578, 46.6329625619284, 46.4642630466512, 46.2947783085965, 46.124539057139, 45.9535752663978, 45.7819161961711, 45.6095904136313, 45.436625815614, 45.2630496513378, 45.0888885460034, 44.9141685244041, 44.7389150351257, 44.5631529749448, 44.3869067135235, 44.2102001180336, 44.033056578126, 43.8554990306471, 43.6775499844879, 43.4992315451591, 43.3205654392693, 43.141573038749, 42.9622753847071, 42.7826932109916, 42.6028469672666, 42.4227568417265, 42.2424427831109, 42.0619245225317, 41.8812215940134, 41.7003533551494, 41.5193390066023, 41.338197610876, 41.1569481105649, 40.9756093456117, 40.7942000696248, 40.612738965748, 40.4312446611179, 40.2497357407831, 40.0682307605659, 39.886748258939, 39.7053067679104, 39.5239248229965, 39.3426209720659, 39.1614137833943, 38.980321852525, 38.7993638080557, 38.6185583164938, 38.4379240861367, 38.2574798697087, 38.0772444662015, 37.8972367217043, 37.7174755286518, 37.5379798252571, 37.3587685924914, 37.1798608510639, 37.0012756574811, 36.8230320981253, 36.6451492838477, 36.4676463423438, 36.2905424108031, 36.1138566262845, 35.9376081166155, 35.7618159898634, 35.5864993231222, 35.4116771499215, 35.2373684479975, 35.0635921258153, 34.8903670082065, 34.7177118215337, 34.5456451786533, 34.3741855626521, 34.2033513105663, 34.0331605964076, 33.8636314136098, 33.6947815572499, 33.5266286061828, 33.359189904064, 33.1924825405686, 33.0265233328095, 32.8613288050688, 32.6969151705301, 32.5332983107341, 32.3704937561831, 32.2085166667816, 32.0473818116364, 31.8871035500673, 31.7276958107315, 31.5691720737643, 31.4115453495206, 31.2548281609458, 31.0990325236347, 30.9441699275286, 30.7902513183616, 30.6372870797201, 30.4852870154166, 30.3342603318288, 30.1842156217969, 30.0351608472998, 29.8871033248128, 29.7400497092285, 29.5940059793534, 29.4489774242828, 29.3049686292371, 29.1619834632994, 29.0200250674648, 28.8790958430174, 28.7391974409087, 28.6003307523848, 28.4624958996029, 28.3256922273249, 28.1899182959219, 28.0551718747954, 27.9214499362308, 27.7887486512205, 27.6570633851431, 27.5263886953046, 27.3967183284367, 27.2680452196776, 27.140361492535, 27.0136584593227, 26.8879266232193, 26.7631556805259, 26.6393345248728, 26.5164512510572, 26.3944931609416, 26.2734467700738, 26.1532978140501, 26.0340312586385, 25.915631307719, 25.7980814128325, 25.6813642869972, 25.5654619128923, 25.4503555593691, 25.3360257930978, 25.2224524928371, 25.1096148671421, 24.9974914685147, 24.8860602113963, 24.7752983899493, 24.6651826963329, 24.5556892405953, 24.4467935706965, 24.3384706936794, 24.2306950964616, 24.123440766385, 24.0166812189625, 23.9103895152824, 23.8045382900144, 23.6990997732283, 23.5940458185916, 23.4893479239344, 23.384977260636, 23.2809046995809, 23.1771008363197, 23.0735360185839, 22.9701803744193, 22.86700383743, 22.7639761770412, 22.6610670234884, 22.5582459000344, 22.4554822457362, 22.3527454501178, 22.2500048741434, 22.1472298837208, 22.044389878275, 21.9414543156024, 21.8383927410811, 21.7351748188903, 21.6317703548015, 21.5281493287434, 21.4242819165402, 21.3201385236674, 21.2156898084583, 21.1109067067967, 21.0057604626116, 20.9002226505743, 20.7942652031427, 20.6878604335842, 20.5809810638996, 20.4736002435664, 20.3656915774251, 20.2572291441268, 20.1481875242227, 20.0385418175576, 19.928267661817, 19.8173412593452, 19.7057393888518, 19.5934394327343, 19.480419387904, 19.3666578880535, 19.2521342152399, 19.1368283183289, 19.0207208294364, 18.9037930761865, 18.7860270937048, 18.6674056390379, 18.5479122008711, 18.427531012077, 18.306247060196, 18.1840460907296, 18.0609146206617, 17.9368399430648, 17.8118101349129, 17.6858140584464, 17.5588413749695, 17.430882533268, 17.3019287847469, 17.1719721784996, 17.0410055700548, 16.9090226017309, 16.7760177332238, 16.6419862081248, 16.5069240672639, 16.3708281474252, 16.2336960643258, 16.0955262229784, 15.9563177952843, 15.8160707265999, 15.6747857207449, 15.5324642346944, 15.3891084642473, 15.2447213448447, 15.0993065262141, 14.9528683754646, 14.8054119562214, 14.656943018533, 14.5074679858558, 14.3569939429107, 14.2055286157954, 14.0530803673318, 13.8996581674523, 13.7452715968429, 13.5899308097149, 13.4336465329305, 13.2764300392984, 13.118293139952, 12.9592481571719, 12.7993079079553, 12.6384857000456, 12.4767952898823, 12.3142508797318, 12.1508671023058, 11.986658986029, 11.8216419568848, 11.655831795626, 11.4892446455962, 11.3218969695187, 11.1538055430917, 10.9849874469538, 10.8154600178777, 10.645240858891, 10.4743478105924, 10.3027989343744, 10.1306124832709, 9.95780691320687, 9.78440082569668, 9.61041299356312, 9.43586231291834, 9.26076779466312, 9.08514856162548, 8.90902381893942, 8.73241284596297, 8.5553349806249, 8.37780960605336, 8.19985613974069, 8.02149401083766, 7.84274265512935, 7.66362151857757, 7.48415002494955, 7.30434756332665, 7.12423349639371, 6.94382714307679, 6.76314777330453, 6.5822145964378, 6.40104675961599, 6.21966332153271, 6.03808328134374, 5.85632554976169, 5.6744089615962, 5.49235224693472, 5.31017405053636, 5.1278929222616, 4.94552733383172, 4.7630956438623, 4.58061611392994, 4.39810692696861, 4.21558618436176, 4.03307186459168, 3.85058189164467, 3.66813409511779, 3.48574622114505, 3.30343597714596, 3.12122095689142, 2.93911873160892, 2.75714680156329, 2.57532264063189, 2.39366365567674, 2.21218725815287, 2.03091082847765, 1.84985172047949, 1.66902731392916, 1.48845499694117, 1.30815216053491, 1.12813624260087, 0.948424745931682, 0.769035190681329, 0.589985191602977, 0.411292445811938, 0.232974718181722, 0.05504993034824, -0.122463931536466, -0.299548714150056, -0.476186119305133, -0.652357705632767, -0.828044861163882, -1.00322875728213, -1.17789042096104, -1.35201061511198, -1.52556990515409, -1.69854863727642, -1.87092688100508, -2.04268444818962, -2.21380092548069, -2.38425553857691, -2.55402731813071, -2.72309489582594, -2.8914366550959, -3.05903064535035, -3.22585457602585, -3.39188582378199, -3.55710142426637, -3.72147805387473, -3.88499202682281, -4.04761932152248, -4.20933551817409, -4.3701158497493, -4.52993514594969, -4.68876788349219, -4.84658817767994, -5.00336973243024, -5.15908586132954, -5.31370957178989, -5.46721343446205, -5.61956968277372, -5.77075017729436, -5.92072642551337, -6.06946957132991, -6.21695042681512, -6.36313947988516, -6.5080068566642, -6.65152241572176, -6.79365568291514, -6.93437589243818, -7.07365205558102, -7.21145284181858, -7.34774675601629, -7.48250204525723, -7.61568674918563, -7.74726873401318, -7.8772156858991, -8.00549515809284, -8.13207462364711, -8.2569213697406, -8.38000267668679, -8.5012857901929, -8.62073789696094, -8.73832620671824, -8.85401801357841, -8.96778060123645, -9.07958139421026, -9.18938794926774, -9.2971679939919, -9.40288942375487, -9.5065203851227, -9.60802926000604, -9.70738475146095, -9.80455587370614, -9.89951205036068, -9.99222302731315, -10.0826590430386, -10.1707908560709, -10.2565896419178, -10.3400271670006, -10.4210758113877, -10.4997085505241, -10.5758989942969, -10.6496215430917, -10.7208512479715, -10.7895638859116, -10.85573620913, -10.9193457308321, -10.9803707508149, -11.0387906278645, -11.0945856950867, -11.1477370876956, -11.1982271147376, -11.246039115546, -11.2911575582479, -11.3335678887234, -11.3732568664108, -11.4102123462289, -11.4444233862281, -11.4758803244423, -11.5045747686444, -11.5304996471872, -11.5536490208579, -11.5740185868442, -11.5916051146938, -11.6064069456455, -11.6184237606033, -11.6276566021479, -11.6341080108912, -11.6377819814498, -11.6386839111844, -11.6368207535121, -11.6322007430076, -11.6248338988572, -11.6147315119068, -11.6019063487573, -11.5863728150275, -11.5681466707021, -11.5472452321988, -11.5236873442035, -11.4974931609563, -11.4686844647374, -11.4372844765508, -11.4033177927453, -11.3668104491975, -11.3277899589826, -11.2862851678283, -11.2423262067973, -11.1959447938187, -11.1471736844392, -11.0960470376562, -11.04260042854, -10.9868703129239, -10.9288947873448, -10.868712745149, -10.806364314885, -10.7418906863129, -10.6753342466185, -10.6067381765229, -10.5361466447707, -10.4636048101383, -10.3891586670486, -10.3128548799359, -10.2347411977527, -10.1548657015455, -10.0732772067137, -9.99002538965787, -9.90516012662395, -9.81873188661864, -9.73079171174578, -9.64139075248428, -9.55058069690057, -9.45841330976496, -9.36494056406289, -9.27021464187384, -9.17428750883225, -9.0772114457879, -8.97903859235084, -8.87982077814301, -8.77960970968291, -8.67845702972446, -8.57641371078229, -8.4735304189474, -8.36985762599353, -8.26544494952283, -8.16034184300907, -8.05459673880993, -7.94825726933367, -7.84137089176569, -7.73398377497945, -7.62614129010854, -7.5178879855759, -7.40926734193585, -7.30032206077717, -7.19109315452225, -7.08162137994454, -6.97194568834109, -6.86210385765521, -6.75213266146668, -6.64206754051421, -6.53194228896793, -6.42178960159675, -6.31164112113365, -6.20152602381721, -6.09147321617765, -5.98150916083386, -5.87165935092734, -5.76194778385916, -5.65239654601928, -5.54302656787712, -5.4338566684498, -5.32490492313265, -5.2161870226182, -5.10771757226878, -4.99950938916976, -4.89157360458493, -4.78392030728489, -4.67655752911637, -4.56949188649956, -4.462728582564, -4.35627111747313, -4.25012197680903, -4.14428146800093, -4.03874945162573, -3.93352325652487, -3.82860012498787, -3.72397477188464, -3.61964160850273, -3.51559348940319, -3.41182167673541, -3.30831707058555, -3.20506902739083, -3.10206579225718, -2.99929509601287, -2.8967434332333, -2.79439656473806, -2.69223931977675, -2.59025621899111, -2.48843084310604, -2.38674605387356, -2.28518472333212, -2.18372853813625, -2.0823594682473, -1.98105880916948, -1.87980796975335, -1.77858749770655, -1.67737905374488, -1.57616306382678, -1.47492089341136, -1.37363387778147, -1.27228226020141, -1.1708491769462, -1.06931583453171, -0.967664505796776, -0.865878195489998, -0.763940695044539, -0.661835340520447, -0.559547342377826, -0.457062214566697, -0.354365580136964, -0.251445063422643, -0.148288448765918, -0.044884926712634, 0.05877655899566, 0.162704358353096, 0.266906475951757, 0.371390048745397, 0.476160641605681, 0.581223522319966, 0.686581629371972, 0.792237243952393, 0.898191504287986, 1.00444438563322, 1.11099360849187, 1.21783769721466, 1.32497173758436, 1.43239101128532, 1.54008955443113, 1.64805869243957, 1.75629063447526, 1.86477522588751, 1.97350164132982, 2.08245715800966, 2.191629358259, 2.30100377000926, 2.41056561233142, 2.52029808734559, 2.63018562976694, 2.7402095846051, 2.85035218607537, 2.96059382604762, 3.07091625869036, 3.18129888741424, 3.29172138714915, 3.40216294993881, 3.51260279270881, 3.62302003290111, 3.73339394413636, 3.84370291163716, 3.95392693970595, 4.06404429802967, 4.17403626140656, 4.28388256153878, 4.39356368497093, 4.5030617658044, 4.61235887563981, 4.72143909766476, 4.83028539326687, 4.93888346043814, 5.04722051657618, 5.15528404074328, 5.26306298800466, 5.370548312499, 5.47773278371318, 5.58460972509316, 5.69117468145214, 5.79742577551124, 5.90336315544088, 6.00898747160013, 6.11430203585008, 6.21931283040847, 6.32402802635511, 6.42845649003845, 6.53261188842446, 6.63650796517665, 6.74016183592362, 6.84359343024533, 6.9468226261119, 7.04987457866186, 7.15277418880496, 7.2555516547224, 7.3582371697105, 7.46086274436228, 7.56346571856155, 7.66608086120288, 7.7687494319897, 7.87151292820467, 7.97441540401925, 8.07750065637627, 8.18081650988861, 8.28441235482019, 8.38833711263489, 8.49264402138893, 8.59738532652237, 8.70261455665695, 8.80838777741923, 8.91475982407959, 9.02178821606248, 9.12952889705391, 9.2380399854097, 9.34737879627972, 9.457601506683, 9.56876643485714, 9.68092963307768, 9.79414566617931, 9.90847164486435, 10.0239599542258, 10.1406615332784, 10.2586309050945, 10.377913548956, 10.4985584898048, 10.6206105258621, 10.7441125729377, 10.8691041353587, 10.9956208361733, 11.123699986863, 11.2533727271839, 11.3846646767257, 11.5176013872194, 11.6522026563075, 11.7884879495091, 11.9264661465955, 12.0661505179891, 12.2075445825606, 12.3506488065198, 12.4954585955447, 12.6419677025631, 12.7901627083962, 12.9400267926921, 13.0915398445234, 13.2446768621374, 13.3994038517424, 13.555690416687, 13.7134975398917, 13.8727817629757, 14.033496579459, 14.1955939555781, 14.3590146415521, 14.5237049803857, 14.6896027953538, 14.8566442359189, 15.0247609779272, 15.1938852211535, 15.3639449661365, 15.5348641194021, 15.7065696392897, 15.8789825886139, 16.0520273545357, 16.2256233544517, 16.3996912742479, 16.5741550238995, 16.7489328432587, 16.9239509634978, 17.0991303223324, 17.2743998193609, 17.4496835879712, 17.6249141339457, 17.8000231581895, 17.9749470160532, 18.1496269108644, 18.3240049691527, 18.4980263112237, 18.6716528438187, 18.8448308840986, 19.0175294383561, 19.1897170297097, 19.3613614310878, 19.5324487742006, 19.7029593632621, 19.8728822924626, 20.0422184361551, 20.2109683097617, 20.3791379241919, 20.5467405879679, 20.7138049773456, 20.8803434961927, 21.0463923885131, 21.2119841956298, 21.377154254769, 21.5419495357549, 21.7064092621127, 21.8705867842093, 22.0345288344049, 22.1982926565793, 22.3619192110238, 22.5254693566404, 22.6889922200584, 22.8525361342789, 23.0161461137068, 23.1798667384819, 23.3437403596401, 23.5077957427491, 23.6720661785226, 23.8365670754723, 24.0013150207923, 24.1663172506395, 24.3315651809754, 24.4970491171427, 24.6627407949389, 24.8286063935923, 24.9946061278058, 25.1606790239171, 25.3267526827772, 25.4927538763263, 25.6585891114956, 25.824163424772, 25.9893648403371, 26.1540697564719, 26.3181568398015, 26.4814918059697, 26.6439458127353, 26.8053773567426, 26.9656566689579, 27.1246322113698, 27.2821820059773, 27.4381938610969, 27.5925441054073, 27.7451549373677, 27.8959403023613, 28.0448584532385, 28.1918775089191, 28.3369986733044, 28.4802701255132, 28.6217696032634, 28.7616127800423, 28.8999681984926, 29.0370452395345, 29.1731059552542, 29.3084502182058, 29.4434502770832, 29.5785013808304, 29.7140532652199, 29.8505917209801, 29.9886445025412, 30.1287268499086, 30.2713900802234, 30.4171596720886, 30.5665174718451, 30.7198924134105, 30.8776082859548, 31.0398283563872, 31.2065785069712, 31.3775883855589, 31.5523081880348, 31.729770122417, 31.9085836153856, 32.086711617527, 32.2614318859223, 32.4292130044484, 32.585492127955, 32.7245897410906, 32.8393931662694, 32.9212552716051, 32.9596352342682, 32.9419007778395]
wx = wx[::20]
wy = wy[::20]
# rad = 10
# wx = rad*np.cos(np.linspace(0, 2 * np.pi, 40))-rad
# wy = rad*np.sin(np.linspace(0, 2 * np.pi, 40))

# obstacle lists
ob = np.array([[20.0, 45.0]])

tx, ty, tyaw, tc, csp = generate_target_course(wx, wy)

# initial state
c_speed = 0  # current speed [m/s]
c_acc = 0  # current acceleration [m/ss]
c_d = 0.0  # current lateral position [m]
c_d_d = 0.0  # current lateral speed [m/s]
c_d_dd = 0.0  # current latral acceleration [m/s]
s0 = 0.0  # current course position

area = 30.0  # animation area length [m]

t = 0

if show_animation:
    # initialise the graph and settings
    fig = plt.figure()
    ax = fig.gca()
    fig.show()
    fig.canvas.draw()

a_l = []
sp = []
cost = []
t_arr = []

for i in tqdm(range(SIM_LOOP)):
    path = calc_optimal_path(c_speed, c_acc, c_d, c_d_d, c_d_dd, s0, ob, csp)

    ind = 1
    s0 = path.s[ind]
    c_d = path.d[ind]
    c_d_d = path.d_d[ind]
    c_d_dd = path.d_dd[ind]
    c_speed = path.s_d[ind]
    c_acc = path.s_dd[ind]
    t += path.t[ind]

    if len(path.c) > ind:
        a_l.append(path.s_d[ind] ** 2 * path.c[ind])
        sp.append(path.s_d[ind])
        cost.append(path.cf)

        t_arr.append(t)
    else:
        break

    if path.cf == np.inf:
        print("No feasible path!")
        break

    if np.hypot(path.x[1] - tx[-1], path.y[1] - ty[-1]) <= 1.0:
        print("Goal")
        break

    if show_animation:  # pragma: no cover
        ax.clear()
        # for stopping simulation with the esc key.
        ax.set_aspect('equal', adjustable='box')
        ax.plot(wx, wy, '.')
        ax.plot(tx, ty)
        ax.plot(ob[:, 0], ob[:, 1], "xk")
        ax.plot(path.x[1:], path.y[1:], "-or")
        ax.plot(path.x[1], path.y[1], "vc")
        plt.xlim(path.x[1] - area, path.x[1] + area)
        plt.ylim(path.y[1] - area, path.y[1] + area)
        plt.title("v[m/s]:" + str(c_speed)[0:4] + ' t[s]:' + str(t))
        ax.grid(True)
        fig.canvas.draw()  # draw
        fig.savefig(str(i)+'.png')
    time.sleep(0.01)  # sleep

print("Finish")

plt.figure()
plt.plot(t_arr, cost, '.-')
plt.grid()